# 出站认证 (Outbound Auth)

出站认证允许 agent 和 AgentKit Gateway 代表在入站认证期间已经过身份验证和授权的用户安全地访问火山引擎资源和第三方服务。要将授权与火山引擎资源或第三方服务集成，需要同时配置入站认证和出站认证。

通过 Agent Identity 支持的最小权限访问和安全权限委托，agent可以无缝且安全地访问火山引擎资源和第三方工具，如 飞书、Coze、GitHub 和 Google。agent可以代表用户或独立地在这些服务上执行操作，前提是有预先授权的用户同意。此外，您可以使用安全令牌库减少用户授权疲劳，并创建流畅的 AI Agent 体验。

## 核心概念

### 出站认证配置

首先，您需要向第三方提供商注册您的客户端应用程序，然后创建出站认证。您需要指定如何验证对火山引擎资源或第三方服务或 AgentKit Gateway 目标的访问。您可以使用 OAuth 2LO/3LO 或 API 密钥。对于 OAuth，您可以从 Agent Identity 提供的提供商中选择。在这种情况下，您需要输入来自 Agent Identity 的提供商配置详细信息。或者，您可以提供自定义提供商的详细信息。

当用户想要访问火山引擎资源或第三方服务或 AgentKit Gateway 目标时，出站认证会确认入站认证提供的访问令牌是否有效，如果有效，则允许访问资源。

<div style="text-align:center">
    <img src="images/outbound_auth.png" width="90%"/>
</div>

### 资源凭据提供商

这是agent代码用来检索下游资源服务器（例如 Google、GitHub）凭据以访问它们的组件，例如从 Gmail 获取电子邮件、向 Google 日历添加会议。它消除了agent开发人员在最终用户、agent代码和外部授权服务器之间实现 2LO 和 3LO OAuth2 编排流程的繁重工作。Agent Identity 提供自定义 OAuth2 凭据提供商和内置提供商列表，如 飞书、Coze、GitHub以及火山引擎各产品服务，其中授权服务器端点和提供商特定参数已预填充。
  
Agent Identity 为Agent开发人员提供 OAuth2 和 API 密钥凭据提供商，以便与支持 OAuth2 或 API 密钥的外部资源进行身份验证。在以下示例中，我们将引导您配置 API 密钥凭据提供商。然后agent可以使用 API 密钥凭据提供商为任何agent操作检索 API 密钥。请参阅文档了解其他凭据提供商。

#### 创建资源凭据提供商

以下是创建 API 密钥资源凭据提供商的示例。

```
from veadk.integrations.ve_identity import IdentityClient

identity_client = IdentityClient(region="cn-beijing")

api_key_provider = identity_client.create_api_key_credential_provider({
    "name": "APIKey-provider-name",
    "apiKey": "<my-api-key>" # 替换为您从外部应用程序供应商（例如 OpenAI）获得的 API 密钥
})
print(api_key_provider)
```

#### 从资源凭据提供商检索访问令牌或 API 密钥

以下是从 API 密钥凭据提供商检索 API 密钥的示例。agent可以使用 API 密钥与 LLM 或其他使用 API 密钥配置的服务进行交互。为了从凭据提供商检索凭据（如 access_token 或 API 密钥），您可以按如下所示封装您的函数。

```
import asyncio
from veadk.integrations.ve_identity import APIKeyFunctionTool

async def need_api_key(*, api_key: str):
    print(f'received api key for async func: {api_key}')

need_api_key_tool = APIKeyFunctionTool(
    func=need_api_key,
    provider_name="APIKey-provider"
)

await need_api_key_tool.run_async(args=[], tool_context=None)
```

以下是您可以与 APIKeyFunctionTool 包装类一起使用的各种参数。


| 参数名称             | 描述                                                              |
|:--------------------|:-------------------------------------------------------------------------|
| provider_name       | 凭据提供商名称                                             |
| into                | 要注入令牌的参数名称                                  |
| identity_client              | 可选参数，用于传入初始化好的Agent Identity Client                                                 |
| region           | 可选参数，用于指定Agent Identity服务的区域，默认为cn-beijing，在未提供identity_client的情况下生效                    |
		

## 概述

让我们创建一个 agent 使用OpenWeather的API，并为其配置使用API密钥凭据提供商的出站认证。您将设置一个API密钥凭据提供商来存储OpenWeather API密钥，并修改agent代码以使用此密钥。

**说明**：你可以在[使用 API Key 进行出站认证](https://volcengine.github.io/veadk-python/auth/api-key-outbound/)找到更详细的文档。

### Tutorial 架构

<div style="text-align:center">
    <img src="images/outbound_auth_api.svg" width="90%"/>
</div>


### Tutorial 说明

| 信息                | 详情                                                                  |
|:--------------------|:-------------------------------------------------------------------------|
| Agent类型            | 单Agent                                                                   |
| Agent框架            | Volcengine Agent Development Kit(VeADK) Agents                                                           |
| Tutorial 核心模块            | AgentKit Runtime, Agent Identity |
| 教程复杂度          | 简单                                                                     |
| 使用的 SDK          | AgentKit Python SDK, VeADK, volcengine-python-sdk                             |
| 凭据提供商          | 类型：API 密钥                                                           |


### 教程关键特性

* 在 AgentKit Runtime 上托管Agent
* 使用火山方舟模型
* 使用 AgentKit Python SDK
* 使用 Agent Identity 出站认证与 API 密钥凭据提供商


## 教程

### 先决条件

要执行本教程，您需要：
* Python 3.10+
* Volcengine 凭据
* AgentKit Python SDK 
* VeADK
* [uv](https://docs.astral.sh/uv/#scripts)

In [ ]:
!uv pip install --force-reinstall -U -r requirements.txt -q # 您可以通过 `--index` 参数指定镜像源以加快安装速度。

设置火山引擎 AccessKey 和 SecretKey 以及方舟 API Key 来使用 Agent Identity 接口以及进行 AgentKit Runtime 部署

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
from agentkit.toolkit import sdk

# 检查 .env 文件是否存在
env_file = Path('.env')
env_template = Path('.env.template')

if not env_file.exists():
    if env_template.exists():
        raise FileNotFoundError(
            "未找到 .env 文件！\n"
            "请先复制模板文件：cp .env.template .env\n"
            "然后填写您的配置信息。"
        )
    else:
        raise FileNotFoundError(
            "未找到 .env 文件！请创建 .env 文件并设置以下环境变量：\n"
            "- VOLCENGINE_ACCESS_KEY\n"
            "- VOLCENGINE_SECRET_KEY\n"
            "- MODEL_API_KEY"
        )


load_dotenv()

VOLCENGINE_ACCESS_KEY = os.getenv("VOLCENGINE_ACCESS_KEY")
VOLCENGINE_SECRET_KEY = os.getenv("VOLCENGINE_SECRET_KEY")
OPEN_WEATHER_API_KEY = os.getenv("OPEN_WEATHER_API_KEY")
MODEL_API_KEY = os.getenv("MODEL_API_KEY")

if not VOLCENGINE_ACCESS_KEY:
    raise ValueError("请设置 VOLCENGINE_ACCESS_KEY环境变量")

if not VOLCENGINE_SECRET_KEY:
    raise ValueError("请设置 VOLCENGINE_ACCESS_KEY环境变量")

if not OPEN_WEATHER_API_KEY:
    raise ValueError("请设置 OPEN_WEATHER_API_KEY环境变量")

if not MODEL_API_KEY:
    raise ValueError("请设置 MODEL_API_KEY环境变量")

### 创建您的Agent并在本地进行实验

在将我们的 Agent 部署到 AgentKit Runtime 之前，让我们先在本地开发和运行它们以进行实验。

对于生产级的 Agent 应用程序，我们需要将 Agent 创建过程与 Agent 调用过程解耦。在 AgentKit Runtime 中，我们将使用 `@app.entrypoint` 装饰器来装饰 Agent 的调用部分，并将其作为运行时的入口点。让我们首先看看在实验阶段每个 Agent 是如何开发的。

In [ ]:
!mkdir -p test-agents

In [ ]:
%%writefile test-agents/agent.py
from veadk import Agent, Runner
import argparse
import json
import asyncio

# 创建自定义工具
def weather():
    """ 获取天气 """ # Dummy weather function
    return "晴天"

agent = Agent(
    model_name="doubao-seed-1-6-250615",
    tools=[weather],
    instruction="您是一个有用的助手。您可以告诉我当前的天气情况。"
)
runner = Runner(agent=agent)

async def run(payload: dict, headers: dict) -> str:
    prompt = payload["prompt"]
    if prompt is None:
        return "prompt is None"

    user_id = headers.get("user_id", "testuser")
    session_id = headers.get("session_id", "testsession")

    response = await runner.run(
        messages=prompt,
        user_id=user_id,
        session_id=session_id
    )
    return response

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = asyncio.run(run(json.loads(args.payload), {}))
    print(response)

#### 调用本地Agent

In [ ]:
!uv run test-agents/agent.py '{"prompt": "现在的天气怎么样？"}'

### 创建资源凭据提供商

In [ ]:
from veadk.integrations.ve_identity import IdentityClient
from volcenginesdkcore.rest import ApiException

#配置 API 密钥提供商
identity_client = IdentityClient(region="cn-beijing")

try:
    api_key_provider = identity_client.create_api_key_credential_provider({
        "name": "weather-apikey-provider",
        "api_key": OPEN_WEATHER_API_KEY
    })
    print(api_key_provider)
except ApiException as e:
    if "Duplicate entry" in e.reason:
        print("weather-apikey-provider already exists")
    else:
        print(e.reason)


### 准备您的 Agent 以部署到 AgentKit Runtime 并使用资源凭据提供商

现在让我们将 Agent 部署到 AgentKit Runtime。为此我们需要：
* 导入 Runtime App：使用 from agentkit.apps import AgentkitSimpleApp 导入 AgentKit 应用
* 在代码中初始化 App：使用 app = AgentkitSimpleApp() 初始化应用
* 装饰调用函数：使用 @app.entrypoint 装饰器装饰调用函数
* 让 AgentKit Runtime 控制 Agent 运行：使用 app.run() 启动应用
* 从资源凭据提供商检索模型 API 密钥：从前面步骤创建的资源凭据提供商中获取 OpenWeather API 访问密钥

#### 在 VeADK Agent 中使用托管的凭据访问OpenWeather的付费 API
让我们从替换agent中的weather工具实现开始，所有其他Agent的工作方式完全相同。

In [ ]:
%%writefile test-agents/agent.py

import json
import requests
from agentkit.apps import AgentkitSimpleApp

from veadk import Agent, Runner
from veadk.integrations.ve_identity import VeIdentityFunctionTool, api_key_auth


def weather(lat: float, lon: float, *, api_key: str) -> str:
    """
    Retrieves the weather information for a given location using latitude and longitude.

    Args:
        lat: Latitude of the location (纬度)
        lon: Longitude of the location (经度)
        api_key: OpenWeatherMap API key

    Returns:
        Weather information as a JSON string

    Note:
        - lat (latitude) is the north-south position (纬度)
        - lon (longitude) is the east-west position (经度)
        - For example: Beijing is approximately lat=39.9042, lon=116.4074
    """
    try:
        print(f"Getting weather for coordinates: lat={lat}, lon={lon}")
        print(f"Using API key: {api_key[:8]}...")

        # OpenWeatherMap Weather API 2.5
        url = f"https://api.openweathermap.org/data/2.5/weather"
        params = {
            "lat": lat,
            "lon": lon,
            "appid": api_key,
            "units": "metric",  # 使用摄氏度
            "lang": "zh_cn",  # 中文描述
        }

        response = requests.get(url, params=params)
        response.raise_for_status()

        weather_data = response.json()

        # 提取关键天气信息
        main_data = weather_data.get("main", {})
        weather_info = {
            "location": f"纬度: {lat}, 经度: {lon}",
            "current_temperature": f"{main_data.get('temp', 'N/A')}°C",
            "feels_like": f"{main_data.get('feels_like', 'N/A')}°C",
            "humidity": f"{main_data.get('humidity', 'N/A')}%",
            "weather_description": weather_data.get("weather", [{}])[0].get(
                "description", "N/A"
            ),
            "wind_speed": f"{weather_data.get('wind',{}).get('speed', 'N/A')} m/s",
            "pressure": f"{main_data.get('pressure', 'N/A')} hPa",
        }

        return json.dumps(weather_info, ensure_ascii=False, indent=2)

    except requests.exceptions.RequestException as e:
        return f"Error fetching weather data: {str(e)}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"


weather_tool = VeIdentityFunctionTool(
    func=weather,
    auth_config=api_key_auth(
        provider_name="openweather-provider",
    ),
)

agent = Agent(
    model_name="doubao-seed-1-6-250615",
    tools=[weather_tool],
    instruction="""您是一个有用的天气助手。当用户询问某个城市的天气时，您需要：

1. 根据城市名称确定其经纬度坐标
2. 使用 weather 函数查询天气，传入正确的 lat（纬度）和 lon（经度）参数

重要说明：
- lat 参数是纬度（latitude），表示南北位置
- lon 参数是经度（longitude），表示东西位置

常见城市的经纬度参考：
- 北京：lat=39.9042, lon=116.4074
- 上海：lat=31.2304, lon=121.4737
- 广州：lat=23.1291, lon=113.2644
- 深圳：lat=22.5431, lon=114.0579
- 杭州：lat=30.2741, lon=120.1551
- 成都：lat=30.5728, lon=104.0668
- 西安：lat=34.3416, lon=108.9398
- 武汉：lat=30.5928, lon=114.3055

如果用户询问的城市不在上述列表中，请根据您的知识估算其大致经纬度坐标。""",
)
runner = Runner(agent=agent)
app = AgentkitSimpleApp()


@app.entrypoint
async def run(payload: dict, headers: dict) -> str:
    prompt = payload["prompt"]
    if prompt is None:
        return "prompt is None"

    user_id = headers.get("user_id", "testuser")
    session_id = headers.get("session_id", "testsession")

    response = await runner.run(messages=prompt, user_id=user_id, session_id=session_id)
    return response


@app.ping
def ping() -> str:
    return "pong!"


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)


### 幕后发生了什么？

当您使用 `AgentkitSimpleApp` 时，它会自动：

* 创建一个监听端口 8000 的 HTTP 服务器
* 实现用于处理 Agent 请求的 `/invoke` 端点
* 实现用于健康检查的 `/ping` 端点（对异步 Agent 非常重要）
* 处理适当的内容类型和响应格式
* 根据 AgentKit 标准管理错误处理

### 将 Agent 部署到 AgentKit Runtime

AgentKit Platform 的 `CreateRuntime` 操作支持全面的配置选项，让您可以指定容器镜像、环境变量和加密设置。您还可以配置协议设置（HTTP、MCP）和授权机制，以控制客户端与 Agent 的通信方式。

注意： 操作最佳实践是将代码打包为容器，结合火山引擎镜像仓库产品，通过 CI/CD 流水线和基础设施即代码（IaC）将镜像推送到容器镜像仓库。

在本教程中，我们将使用 AgentKit Python SDK 轻松打包您的代码并将其部署到 AgentKit Runtime 上。

#### 配置 AgentKit Runtime 部署

接下来，我们将使用 AgentKit SDK 来配置 AgentKit Runtime 部署，包括EntryPoint、用于访问火山方舟模型的API密钥以及运行的Python版本。

在配置步骤中，将根据您的应用程序代码自动生成 agentkit.yaml 配置文件，它用于在本地保存AgentKit Runtime的配置。

**说明** - 在本教程中使用 cloud 部署模式，即在远程构建镜像并自动部署到AgentKit Runtime上

In [ ]:
import os
from pathlib import Path
from agentkit.toolkit.sdk.config import AgentConfig

current_dir = os.getcwd()
# 如果当前目录不是 test-agents，则切换到该目录
if not current_dir.endswith("test-agents"):
    os.chdir(os.path.join(current_dir, "test-agents"))
agent_file = Path("agentkit.yaml")
if not agent_file.exists():
    agent_file.write_text("")
    print(f"Created empty file: {agent_file}")
else:
    print(f"File already exists, skipping: {agent_file}")

# 指定配置文件路径
config = AgentConfig("agentkit.yaml") \
    .set("common.agent_name", "veidentity_outbound_agent") \
    .set("common.entry_point", "agent.py") \
    .set("common.language", "Python") \
    .set("common.language_version", "3.12") \
    .set("common.launch_type", "cloud") \
    .set("launch_types.cloud.region", "cn-beijing") \
    .set("launch_types.cloud.runtime_id", "") \
    .set("launch_types.cloud.runtime_name", "") \
    .set("launch_types.cloud.runtime_role_name", "") \
    .set("launch_types.cloud.runtime_envs.MODEL_AGENT_API_KEY", MODEL_API_KEY) \
    .save()

print("Configuration created and saved!")

#### 检查 AgentKit 配置

In [ ]:
!cat agentkit.yaml

#### 启动 AgentKit Runtime

现在我们有了配置文件，让我们将 Agent 启动到 AgentKit Runtime。这将会：

🔨 构建阶段 (Build Phase)
- 生成 Dockerfile - 根据项目配置自动生成容器化配置
- 构建 Docker 镜像 - 将你的 Agent 代码打包成容器镜像
- 准备镜像仓库 - 创建火山引擎 Container Registry (CR) 实例和命名空间
- 推送镜像 - 将构建好的镜像上传到云端镜像仓库

🚀 部署阶段 (Deploy Phase)
- 创建 IAM 角色 - 为 Runtime 生成必要的权限角色
- 创建 Runtime 实例 - 在火山引擎上创建新的 AgentKit Runtime
- 配置认证 - 设置 API Key 或 JWT 认证方式
- 等待就绪 - 监控 Runtime 状态直到达到 Ready 状态
- 获取访问端点 - 返回可用的公网访问地址

In [ ]:
launch_result = sdk.launch()
launch_result

#### 调用 AgentKit Runtime

最后，我们可以使用agent调用我们的 AgentKit Runtime

In [ ]:
invoke_response = sdk.invoke(
    payload={
        "prompt": "今天杭州天气怎么样？",
    }
)
invoke_response

#### 处理调用结果

我们现在可以处理调用结果以将其包含在应用程序中

In [ ]:
from IPython.display import Markdown, display
response_text = invoke_response.response
display(Markdown(response_text))

### 清理（可选）

- 现在让我们清理创建的 AgentKit Runtime

In [ ]:
destroy_result = sdk.destroy()
destroy_result

## 总结

您已成功完成了本教程！您学会了如何：

1. 配置 API 密钥凭据提供商
2. 创建使用火山方舟模型的 VeADK Agent
3. 实现出站认证以安全地管理 API 密钥
4. 将agent部署到 AgentKit Runtime
5. 测试具有外部 API 集成的完整工作流程

现在您可以扩展此示例以集成其他服务或构建更复杂的Agent工作流程。